清单13.1 普通区块链

In [2]:
import hashlib
import datetime
import time
import json

INITIAL_BITS = 0x1e777777
MAX_32BIT = 0xffffffff

class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def to_json(self):
        return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
        h = hashlib.sha256(blockheader.encode()).hexdigest()
        self.block_hash = h
        return h
    
    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits
    
    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

class Blockchain():
    def __init__(self, initial_bits):
        self.chain = []
        self.initial_bits = initial_bits

    def add_block(self, block):
        self.chain.append(block)
    
    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))
    
    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = \
                    str((end_time - start_time) / \
                        1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "创世区块", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)
    
    def add_newblock(self, i):
        last_block = self.chain[-1]
        block = Block(i+1, last_block.block_hash, "区块 " + str(i+1), datetime.datetime.now(), last_block.bits)
        self.mining(block)

if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("创世区块创建中……")
    bc.create_genesis()
    for i in range(30):
        print("正在创建第" + str(i+2) + "个区块……")
        bc.add_newblock(i)

创世区块创建中……
{
  "bits": "1e777777",
  "block_hash": "000026dc4ac983ab8e5c3bab753ed2355a4efdfc8f665e021723f539b5680242",
  "elapsed_time": "0.364秒",
  "index": 0,
  "nonce": "0000c1de",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "创世区块",
  "timestamp": "2019/10/09 11:55:03"
}
正在创建第2个区块……
{
  "bits": "1e777777",
  "block_hash": "00001a5bcfeea13015d76630b43c9a1293723d9d71ba7b8d40707602c1009779",
  "elapsed_time": "0.056秒",
  "index": 1,
  "nonce": "00002163",
  "prev_hash": "000026dc4ac983ab8e5c3bab753ed2355a4efdfc8f665e021723f539b5680242",
  "stored_data": "区块1",
  "timestamp": "2019/10/09 11:55:04"
}
正在创建第3个区块……
{
  "bits": "1e777777",
  "block_hash": "00003046fe54af32a4963c140aadc16b06fdd077dfe05540c9463c486ce16595",
  "elapsed_time": "0.341秒",
  "index": 2,
  "nonce": "0000c784",
  "prev_hash": "00001a5bcfeea13015d76630b43c9a1293723d9d71ba7b8d40707602c1009779",
  "stored_data": "区块2",
  "timestamp": "2019/10/09 11:55:04"
}
正在创建第4个区

{
  "bits": "1e777777",
  "block_hash": "0000424961ae7d1ff92d2487c7eb3c748b1bd4d5641bce929b42ae175e095e6c",
  "elapsed_time": "1.606秒",
  "index": 25,
  "nonce": "0003a264",
  "prev_hash": "000059ca2afbd7bae5c8bc89759ff55064f9f7c9a4a4ac793229a8624e1b2e75",
  "stored_data": "区块 25",
  "timestamp": "2019/10/09 11:55:31"
}
正在创建第27个区块……
{
  "bits": "1e777777",
  "block_hash": "00000cb20600878b4fd724a978b90b91dee715361b7c7f2361024f300ecb5529",
  "elapsed_time": "0.025秒",
  "index": 26,
  "nonce": "00000fa9",
  "prev_hash": "0000424961ae7d1ff92d2487c7eb3c748b1bd4d5641bce929b42ae175e095e6c",
  "stored_data": "区块26",
  "timestamp": "2019/10/09 11:55:33"
}
正在创建第28个区块……
{
  "bits": "1e777777",
  "block_hash": "0000693b12a0e8074b56d34a900aea89389389563648dceedd6dd2b6626829a1",
  "elapsed_time": "1.796秒",
  "index": 27,
  "nonce": "0003d18c",
  "prev_hash": "00000cb20600878b4fd724a978b90b91dee715361b7c7f2361024f300ecb5529",
  "stored_data": "区块 27",
  "timestamp": "2019/10/09 11:55:33"
}
正在创建第29个区

清单13.2 清单 13.1 的输出结果（JSON 格式）

清单13.3　清单13.1的INITIAL_BITS 值改为1d777777 的输出结果（JSON格式）

In [4]:
import hashlib
import datetime
import time
import json

INITIAL_BITS = 0x1d777777
MAX_32BIT = 0xffffffff

class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def to_json(self):
        return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
        h = hashlib.sha256(blockheader.encode()).hexdigest()
        self.block_hash = h
        return h
    
    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits
    
    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

class Blockchain():
    def __init__(self, initial_bits):
        self.chain = []
        self.initial_bits = initial_bits

    def add_block(self, block):
        self.chain.append(block)
    
    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))
    
    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = str((end_time - start_time) / 1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "创世区块", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)
    
    def add_newblock(self, i):
        last_block = self.chain[-1]
        block = Block(i+1, last_block.block_hash, "区块 " + str(i+1), datetime.datetime.now(), last_block.bits)
        self.mining(block)

if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("创世区块创建中……")
    bc.create_genesis()
    for i in range(30):
        print("正在创建第" + str(i+2) + "个区块……")
        bc.add_newblock(i)


创世区块创建中……
{
  "bits": "1d777777",
  "block_hash": "0000000dc763c605cc883b2e5ddc776b8c100be4971de61b522fd9edce595e03",
  "elapsed_time": "93.604 秒",
  "index": 0,
  "nonce": "016cb281",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "创世区块",
  "timestamp": "2019/10/02 18:39:11"
}
正在创建第2个区块……
{
  "bits": "1d777777",
  "block_hash": "0000001b67409cbe40d13edb188772949e33b038014baba694c6b2ce73955761",
  "elapsed_time": "209.994 秒",
  "index": 1,
  "nonce": "03432c25",
  "prev_hash": "0000000dc763c605cc883b2e5ddc776b8c100be4971de61b522fd9edce595e03",
  "stored_data": "区块 1",
  "timestamp": "2019/10/02 18:40:44"
}
正在创建第3个区块……
{
  "bits": "1d777777",
  "block_hash": "000000755bee0428a2f7c565bb366ee43472466106dadc3fe95c5385b11b240c",
  "elapsed_time": "29.004 秒",
  "index": 2,
  "nonce": "00737f07",
  "prev_hash": "0000001b67409cbe40d13edb188772949e33b038014baba694c6b2ce73955761",
  "stored_data": "区块 2",
  "timestamp": "2019/10/02 18:44:14"
}

{
  "bits": "1d777777",
  "block_hash": "00000020ed589d900b80763d48ba5eb2f0cd1a393f04f758b256116785273756",
  "elapsed_time": "228.242 秒",
  "index": 24,
  "nonce": "0386b9e2",
  "prev_hash": "00000064d8cef7d02f0b7168b11afe8dd6ee40a03100e8cc8961c93e80e23c59",
  "stored_data": "区块24",
  "timestamp": "2019/10/02 19:38:10"
}
正在创建第26个区块……
{
  "bits": "1d777777",
  "block_hash": "000000023704c965f304a38c492be312450437363172629cdebe5196856f90b8",
  "elapsed_time": "48.153 秒",
  "index": 25,
  "nonce": "00bde943",
  "prev_hash": "00000020ed589d900b80763d48ba5eb2f0cd1a393f04f758b256116785273756",
  "stored_data": "区块25",
  "timestamp": "2019/10/02 19:41:58"
}
正在创建第27个区块……
{
  "bits": "1d777777",
  "block_hash": "0000001cd952eaff827f84a322e52164d2e7436555bdc793d327d0adf6bc4bb5",
  "elapsed_time": "108.54 秒",
  "index": 26,
  "nonce": "01abe0d8",
  "prev_hash": "000000023704c965f304a38c492be312450437363172629cdebe5196856f90b8",
  "stored_data": "区块26",
  "timestamp": "2019/10/02 19:42:46"
}
正在创建

清单13.4 Block 类的变量

In [ ]:
def __init__(self, index, prev_hash, data, timestamp, bits):
    self.index = index
    self.prev_hash = prev_hash
    self.data = data
    self.timestamp = timestamp
    self.bits = bits
    self.nonce = 0
    self.elapsed_time = ""
    self.block_hash = ""

清单13.5 特殊方法 setitem

In [ ]:
def __setitem__(self, key, value):
    setattr(self, key, value)

清单13.6 to_json 方法

In [ ]:
def to_json(self):
return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

清单13.7 calc_blockhash 方法

In [ ]:
def calc_blockhash(self):
    blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
    h = hashlib.sha256(blockheader.encode()).hexdigest()
    self.block_hash = h
    return h

清单13.8 calc_target 方法

In [ ]:
def calc_target(self):
    exponent_bytes = (self.bits >> 24) - 3
    exponent_bits = exponent_bytes * 8
    coefficient = self.bits & 0xffffff
    return coefficient << exponent_bits

清单13.9 check_valid_hash 方法

In [ ]:
def check_valid_hash(self):
    return int(self.calc_blockhash(), 16) <= self.calc_target()

清单13.10 Blockchain 类的变量

In [ ]:
def __init__(self, initial_bits):
    self.chain = []
    self.initial_bits = initial_bits

清单13.11add_block 方法

In [ ]:
def add_block(self, block):
    self.chain.append(block)

清单13.12 getblockinfo 方法

In [ ]:
def getblockinfo(self, index=-1):
    return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))


清单13.13 mining 方法

In [ ]:
def mining(self, block):
    start_time = int(time.time() * 1000)
    while True:
        for n in range(MAX_32BIT + 1):
            block.nonce = n
            if block.check_valid_hash():
                end_time = int(time.time() * 1000)
                block.elapsed_time = \
                str((end_time - start_time) / \
                    1000.0) + "秒"
                self.add_block(block)
                self.getblockinfo()
                return
        new_time = datetime.datetime.now()
        if new_time == block.timestamp:
            block.timestamp += datetime.timedelta(seconds=1)
        else:
            block.timestamp = new_time

清单13.14 create_genesis 方法

In [ ]:
def create_genesis(self):
    genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "创世区块", datetime.datetime.now(), self.initial_bits)
    self.mining(genesis_block) 


清单13.15 add_newblock 函数

In [ ]:
def add_newblock(self, i):
    last_block = self.chain[-1]
    block = Block(i+1, last_block.block_hash, "区块" + str(i+1), datetime.datetime.now(), last_block.bits)
    self.mining(block)

清单13.16  Blockchain 类的实例化

In [ ]:
if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("创建创世区块中……")
    bc.create_genesis()
    for i in range(30):
        print(“创建第”+str(i+2) + "个区块……")
        bc.add_newblock(i) 